# Prepare genome-scale perturb-seq data

William Colgan May 10 2023

### Setup

In [1]:
import numpy as np
import scanpy as sc

In [50]:
norm = sc.read_h5ad("./data/K562_gwps_normalized_bulk_01.h5ad")

In [52]:
norm

AnnData object with n_obs × n_vars = 11258 × 8248
    obs: 'UMI_count_unfiltered', 'num_cells_unfiltered', 'num_cells_filtered', 'control_expr', 'fold_expr', 'pct_expr', 'core_control', 'mean_leverage_score', 'std_leverage_score', 'energy_test_p_value', 'anderson_darling_counts', 'mann_whitney_counts', 'z_gemgroup_UMI', 'mitopercent', 'TE_ratio', 'cnv_score_z'
    var: 'gene_name', 'mean', 'std', 'cv', 'in_matrix', 'gini', 'clean_mean', 'clean_std', 'clean_cv'

### Remove weak perturbations

In [13]:
# Remove cell with anderson_darling_counts <50 and num_cells_filtered <25 and fold_expr  < .70
filtered = norm[norm.obs['anderson_darling_counts'] >= 50]
filtered = filtered[filtered.obs['num_cells_filtered'] >= 25]
filtered = filtered[filtered.obs['fold_expr'] <= .70]

In [40]:
filtered

View of AnnData object with n_obs × n_vars = 1946 × 8248
    obs: 'UMI_count_unfiltered', 'num_cells_unfiltered', 'num_cells_filtered', 'control_expr', 'fold_expr', 'pct_expr', 'core_control', 'mean_leverage_score', 'std_leverage_score', 'energy_test_p_value', 'anderson_darling_counts', 'mann_whitney_counts', 'z_gemgroup_UMI', 'mitopercent', 'TE_ratio', 'cnv_score_z'
    var: 'gene_name', 'mean', 'std', 'cv', 'in_matrix', 'gini', 'clean_mean', 'clean_std', 'clean_cv'

### Remove low variance genes

Get genes with expression >.25 that are in top 30% of variance

In [34]:
df = filtered.var
std_cutoff = df['std'].quantile(.7)
df = df[df['std'] >= std_cutoff]
df = df[df['mean'] >= 0.25]
var_genes = df.index.values

In [35]:
len(var_genes)

2471

Get top 10 genes for each perturbation

In [36]:
top10 = np.abs(filtered.X).argsort(axis=1)[:, -10:]
top10 = top10.flatten()
top10 = np.unique(top10)
top10_genes = filtered.var.index.values[top10]


In [37]:
len(top10_genes)

2251

Get union of these genes

In [39]:
use_genes = np.unique(np.concatenate((var_genes, top10_genes)))
len(use_genes)

3384

In [42]:
filtered = filtered[:, use_genes]

In [53]:
filtered

View of AnnData object with n_obs × n_vars = 1946 × 3384
    obs: 'UMI_count_unfiltered', 'num_cells_unfiltered', 'num_cells_filtered', 'control_expr', 'fold_expr', 'pct_expr', 'core_control', 'mean_leverage_score', 'std_leverage_score', 'energy_test_p_value', 'anderson_darling_counts', 'mann_whitney_counts', 'z_gemgroup_UMI', 'mitopercent', 'TE_ratio', 'cnv_score_z'
    var: 'gene_name', 'mean', 'std', 'cv', 'in_matrix', 'gini', 'clean_mean', 'clean_std', 'clean_cv'

### Save the filtered data

In [49]:
filtered.obs.to_csv("./data/K562_gwps_obs.csv")
filtered.var.to_csv("./data/K562_gwps_var.csv")
np.savetxt("./data/K562_gwps_norm.csv",filtered.X,delimiter = ',')